In [49]:
import pandas as pd
from predictor import Predictor
from datetime import datetime
import mlflow
import numpy as np
from scipy import stats
import joblib
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

## Run predictions on test set simulation to simulate prediciton of new data

RUN this first
```bash
docker run -p 8002:8080 xgboost_dev
```



In [50]:
def convert_date(x):
    return datetime(x['year'], x['month'], 1)

def convert_remaining_lease(x):
    # example: "60 years 11 months"
    try:
        years = int(x.split(' ')[0])
        months = int(x.split(' ')[2])
    except IndexError:
        # print(x)
        return years * 12
    total_months = years * 12 + months
    return total_months

In [51]:
df = pd.read_csv("../../data/resale_flats_transformed.csv")
df

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,street_name_with_block,postal,x,y,lat,lon,district
0,175972,4,ANG MO KIO,3 ROOM,445,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,54 years 05 months,380000.0,2024,445 ANG MO KIO AVE 10,560445,30492.797320,38779.235406,1.366980,103.855718,20.0
1,175973,4,ANG MO KIO,3 ROOM,345,ANG MO KIO AVE 3,10 TO 12,73.0,New Generation,1978,53 years 01 month,465000.0,2024,345 ANG MO KIO AVE 3,560345,29817.683049,38835.289230,1.367487,103.849652,20.0
2,175974,4,ANG MO KIO,3 ROOM,624,ANG MO KIO AVE 4,01 TO 03,68.0,New Generation,1980,55 years 06 months,370000.0,2024,624 ANG MO KIO AVE 4,560624,28743.810128,40391.591436,1.381562,103.840002,20.0
3,175975,4,ANG MO KIO,4 ROOM,308A,ANG MO KIO AVE 1,19 TO 21,94.0,Model A,2012,87 years 06 months,882888.0,2024,308A ANG MO KIO AVE 1,561308,29198.145518,38613.738122,1.365483,103.844085,20.0
4,175976,4,ANG MO KIO,4 ROOM,260B,ANG MO KIO ST 21,01 TO 03,93.0,Model A,2018,93 years 03 months,765000.0,2024,260B ANG MO KIO ST 21,562260,28070.737039,38987.552958,1.368864,103.833954,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176482,1181,1,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142.0,Apartment,1988,70 years 05 months,587000.0,2017,387 YISHUN RING RD,760387,29553.580030,45697.545797,1.429547,103.847279,27.0
176483,1182,1,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142.0,Apartment,1987,69 years 11 months,590000.0,2017,877 YISHUN ST 81,760877,28237.634702,43967.636908,1.413902,103.835454,27.0
176484,1183,1,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,70 years 01 month,673000.0,2017,836 YISHUN ST 81,760836,27974.684438,44139.027429,1.415452,103.833091,27.0
176485,1184,1,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164.0,Apartment,1992,74 years 05 months,678000.0,2017,633A YISHUN ST 61,761633,28723.547711,44545.762389,1.419131,103.839820,27.0


In [52]:
input_df = pd.read_csv("example.csv")

In [53]:
# take in a part of the actual data we have, run predictions for these then store everything
# (features, pred, actual, time) somewhere for me to use

# 1. get the portion of the actual data 
# simulate pulling from data wareshouse 
df = pd.read_csv("../../data/resale_flats_transformed.csv")

# 2. shape it to something like example.csv 
input_df = pd.read_csv("example.csv")
cols_for_prediction = list(input_df.columns)
cols_for_prediction.append('resale_price')
cols_for_prediction
# need to get remaining_lease_years, remaining_lease_months
df['remaining_lease_months'] = df['remaining_lease'].apply(convert_remaining_lease)
df['remaining_lease_years'] = 0
# get sale date
df['sale_date'] = df[['year', 'month']].apply(convert_date, axis=1)
# separate to test set 
df = df[df['sale_date'] >= datetime(2024, 4, 1)]

# get the columns that are in the input_df (example.csv)
df = df[cols_for_prediction]

# 3. run predictions on this data
predictor = Predictor(port=8002)

# # 4. store the results in a csv file
# this takes quite long 
pred_df = predictor.predict_df(df.drop('resale_price', axis=1))
pred_df.to_csv("predictions.csv", index=False)

Predicted price: $312006.14
Predicted price: $319844.87
Predicted price: $298010.00
Predicted price: $340841.20
Predicted price: $302989.14
Predicted price: $360368.95
Predicted price: $360368.95
Predicted price: $396790.31
Predicted price: $368611.54
Predicted price: $552646.87
Predicted price: $243149.41
Predicted price: $356433.75
Predicted price: $335436.49
Predicted price: $350965.10
Predicted price: $313708.16
Predicted price: $329097.64
Predicted price: $356433.75
Predicted price: $350965.10
Predicted price: $350965.10
Predicted price: $352232.44
Predicted price: $328215.28
Predicted price: $306261.32
Predicted price: $349207.71
Predicted price: $338278.44
Predicted price: $344760.02
Predicted price: $347995.26
Predicted price: $361181.82
Predicted price: $344191.43
Predicted price: $331038.17
Predicted price: $352140.65
Predicted price: $329535.27
Predicted price: $309671.72
Predicted price: $356907.93
Predicted price: $503646.44
Predicted price: $435770.64
Predicted price: $39

python -m mlflow server --host 127.0.0.1 --port 8000

## Simulate collection of ground truth some time in future

In [54]:
pred_df_with_ground_truth = pred_df.copy()
pred_df_with_ground_truth['actual_price'] = df['resale_price'].values[:len(pred_df)]

In [74]:
df.head()

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,resale_price
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,380000.0
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,465000.0
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,370000.0
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,882888.0
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,765000.0


In [55]:
pred_df_with_ground_truth.head(5)

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price,actual_price
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,380000.0
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,465000.0
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,370000.0
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,882888.0
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,765000.0


In [56]:
# # randomly generate actual prices based on some formula of the predicted price
# # this is to simulate the actual data we have
# actual_prices = []
# for i, row in df.iterrows():
#     pred_price = row['predicted_price']
#     actual_price = pred_price + np.random.normal(0, 20000)
#     actual_prices.append(actual_price)

# df['actual_price'] = actual_prices
# df

In [57]:
pred_df_with_ground_truth.to_csv("predictions_with_ground_truth.csv", index=False)

## Monitoring

### ground truth evaluation

In [59]:
# check model history performance in mlflow through code
mlflow.set_tracking_uri('http://localhost:8000')
# Get the model history
model_history = mlflow.search_runs()
model_history

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAE,metrics.MSE,metrics.MAPE,params.subsample_freq,...,params.importance_type,params.subsample,params.min_child_weight,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.user,tags.feature,tags.model,tags.mlflow.runName
0,95525df54dc94480badb2c2e398f602c,0,FINISHED,mlflow-artifacts:/0/95525df54dc94480badb2c2e39...,2024-04-11 07:04:45.180000+00:00,2024-04-11 07:04:57.013000+00:00,0.260327,0.141749,0.873252,0,...,split,1.0,0.001,"[{""run_id"": ""95525df54dc94480badb2c2e398f602c""...",LOCAL,c:\Users\valen\anaconda3\lib\site-packages\ipy...,valen,Resale Price Prediction,LightGBM,enchanting-dog-18
1,bfa21874d08f4798a41d7ab63ae82e5c,0,FINISHED,mlflow-artifacts:/0/bfa21874d08f4798a41d7ab63a...,2024-04-11 07:04:20.464000+00:00,2024-04-11 07:04:45.131000+00:00,0.198472,0.072370,0.790721,None,...,None,None,None,"[{""run_id"": ""bfa21874d08f4798a41d7ab63ae82e5c""...",LOCAL,c:\Users\valen\anaconda3\lib\site-packages\ipy...,valen,Resale Price Prediction,XGBoost,debonair-vole-478
2,2cc3764d90ef4609b4e57fcff522004e,0,FINISHED,mlflow-artifacts:/0/2cc3764d90ef4609b4e57fcff5...,2024-03-29 15:37:34.169000+00:00,2024-03-29 15:37:39.549000+00:00,0.250773,0.131307,0.833802,0,...,split,1.0,0.001,"[{""run_id"": ""2cc3764d90ef4609b4e57fcff522004e""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,LightGBM,chill-swan-395
3,08f5fe0400f649b283b222fa850dc091,0,FINISHED,mlflow-artifacts:/0/08f5fe0400f649b283b222fa85...,2024-03-29 15:37:22.144000+00:00,2024-03-29 15:37:34.144000+00:00,0.196303,0.070854,0.782482,None,...,None,None,None,"[{""run_id"": ""08f5fe0400f649b283b222fa850dc091""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,XGBoost,resilient-cub-967
4,a4e061e14928418cbce808a5b90462ec,0,FINISHED,mlflow-artifacts:/0/a4e061e14928418cbce808a5b9...,2024-03-29 13:42:44.513000+00:00,2024-03-29 13:42:49.864000+00:00,0.250773,0.131307,0.833802,0,...,split,1.0,0.001,"[{""run_id"": ""a4e061e14928418cbce808a5b90462ec""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,LightGBM,omniscient-toad-552
5,7eba7d2fd2ae4f019ce155ba1338b95b,0,FINISHED,mlflow-artifacts:/0/7eba7d2fd2ae4f019ce155ba13...,2024-03-29 13:42:31.960000+00:00,2024-03-29 13:42:44.467000+00:00,0.196303,0.070854,0.782482,None,...,None,None,None,"[{""run_id"": ""7eba7d2fd2ae4f019ce155ba1338b95b""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,XGBoost,casual-mule-437
6,b44e97e862f145f19a1f1838f7a79610,0,FINISHED,mlflow-artifacts:/0/b44e97e862f145f19a1f1838f7...,2024-03-29 13:22:25.378000+00:00,2024-03-29 13:22:30.743000+00:00,0.250773,0.131307,0.833802,0,...,split,1.0,0.001,"[{""run_id"": ""b44e97e862f145f19a1f1838f7a79610""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,LightGBM,industrious-carp-854
7,68279395fa044d798337f962a4600287,0,FINISHED,mlflow-artifacts:/0/68279395fa044d798337f962a4...,2024-03-29 13:22:14.933000+00:00,2024-03-29 13:22:25.325000+00:00,0.196303,0.070854,0.782482,None,...,None,None,None,"[{""run_id"": ""68279395fa044d798337f962a4600287""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,XGBoost,masked-bat-604
8,709239b403a44bef87a88a17ef5f4890,0,FINISHED,mlflow-artifacts:/0/709239b403a44bef87a88a17ef...,2024-03-29 13:20:52.053000+00:00,2024-03-29 13:21:05.995000+00:00,0.196303,0.070854,0.782482,None,...,None,None,None,"[{""run_id"": ""709239b403a44bef87a88a17ef5f4890""...",LOCAL,/Users/ivankoh/.local/lib/python3.9/site-packa...,ivankoh,Resale Price Prediction,XGBoost,popular-gnat-419
9,24f757752aa04c9380df54f2a0bd2695,0,FAILED,mlflow-artifacts:/0/24f757752aa04c9380df54f2a0...,2024-03-29 13:14:35.744000+00:00,2024-03-29 13:14:43.578000+00:00,0.196303,0.070854,0.782482,None,...,None,None,None,None,LOCAL,/Users/ivankoh/

In [60]:
def getmodelhist(model_name):
    model_history = mlflow.search_runs()
    model_history = model_history[model_history["tags.model"] == model_name]

    mean_mse = model_history["metrics.MSE"].mean()
    mean_mae = model_history["metrics.MAE"].mean()
    mean_mape = model_history["metrics.MAPE"].mean()

    return {
        "model_name": model_name,
        "metrics" : {
            "mean_mse": mean_mse,
            "mean_mae": mean_mae,
            "mean_mape": mean_mape
        }
    }



In [61]:
getmodelhist("XGBoost")

{'model_name': 'XGBoost',
 'metrics': {'mean_mse': 0.07110626937122871,
  'mean_mae': 0.19666476107049982,
  'mean_mape': 0.7838551752810231}}

In [62]:
df_gt = pd.read_csv("predictions_with_ground_truth.csv")
df_gt

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price,actual_price
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,380000.0
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,465000.0
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,370000.0
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,882888.0
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,765000.0
...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,585000.0
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,673800.0
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,725000.0
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,1080000.0


In [63]:
# import scaler
TARGET_SCALER_PATH = '_scalers/target_scaler.save'
target_scaler = joblib.load(TARGET_SCALER_PATH)

In [64]:
df_gt["actual_price_scaled"] = target_scaler.transform(df_gt['actual_price'].values.reshape(-1, 1))
df_gt["predicted_price_scaled"] = target_scaler.transform(df_gt['predicted_price'].values.reshape(-1, 1))

In [65]:
df_gt

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price,actual_price,actual_price_scaled,predicted_price_scaled
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,380000.0,-0.584652,-0.996796
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,465000.0,-0.069426,-0.949282
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,370000.0,-0.645267,-1.081633
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,882888.0,2.463595,-0.822013
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,765000.0,1.749019,-1.051453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,585000.0,0.657952,-0.331979
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,673800.0,1.196212,-0.692508
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,725000.0,1.506559,-0.806234
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,1080000.0,3.658386,0.796600


In [66]:
# Get performance metrics of new test data after ground truth collected
mape_new = mean_absolute_percentage_error(df_gt['actual_price_scaled'], df_gt['predicted_price_scaled'])
mae_new = mean_absolute_error(df_gt['actual_price_scaled'], df_gt['predicted_price_scaled'])
mse_new = mean_squared_error(df_gt['actual_price_scaled'], df_gt['predicted_price_scaled'])

# Get similar performance metrics during training
mape_train = getmodelhist("XGBoost")['metrics']['mean_mape']
mae_train = getmodelhist("XGBoost")['metrics']['mean_mae']
mse_train = getmodelhist("XGBoost")['metrics']['mean_mse']

In [67]:
# print Train and new, mae,mse,mape
print(f"Train MAPE: {mape_train}, New MAPE: {mape_new}")
print(f"Train MAE: {mae_train}, New MAE: {mae_new}")
print(f"Train MSE: {mse_train}, New MSE: {mse_new}")

Train MAPE: 0.7838551752810231, New MAPE: 4.39818206013086
Train MAE: 0.19666476107049982, New MAE: 1.2541116938346986
Train MSE: 0.07110626937122871, New MSE: 2.349066732184309


In [76]:
# check if deviation by more than 5 percentage points then notify the team
if mape_new > mape_train + 0.05:
    # notify team
    print(f"MAPE of new test data is {100* (mape_train - mape_new)} higher than training data")
# if 5% increase in mae
if (mae_new - mae_train)/ mae_train > 0.05:
    # notify team
    print(f"MAE of new test data is {mae_train - mae_new} higher than training data")
# if 5% increase in mse
if (mse_new - mse_train)/ mse_train > 0.05:
    # notify team
    print(f"MSE of new test data is {mse_train - mse_new} higher than training data")

MAPE of new test data is -361.43268848498366 higher than training data
MAE of new test data is -1.0574469327641987 higher than training data
MSE of new test data is -2.27796046281308 higher than training data


In [110]:
# in html form for table
html_report_ground_truth_eval = f"""
<h3>Model Performance Monitoring</h3>
<h4>Ground Truth Evaluation</h4>

<table>
  <tr>
    <th>Metric</th>
    <th>Training</th>
    <th>Latest</th>
    <th>Change</th>
    <th>% Change</th>
    <th>Threshold</th>
    <th>Drift</th>
  </tr>
  <tr>
    <td>MAPE</td>
    <td>{round(mape_train, 4)}</td>
    <td>{round(mape_new, 4)}</td>
    <td>{round(mape_new - mape_train, 4)}</td>
    <td>{round(100 * (mape_new - mape_train) / mape_train, 4)}</td>
    <td>5%</td>
    <td>{round(100 * (mape_new - mape_train) / mape_train, 4) > 5}</td>
  </tr>
  <tr>
    <td>MAE</td>
    <td>{round(mae_train, 4)}</td>
    <td>{round(mae_new, 4)}</td>
    <td>{round(mae_new - mae_train, 4)}</td>
    <td>{round(100 * (mae_new - mae_train) / mae_train, 4)}</td>
    <td>5%</td>
    <td>{round(100 * (mae_new - mae_train) / mae_train, 4) > 5}</td>
  </tr>
  <tr>
    <td>MSE</td>
    <td>{round(mse_train, 4)}</td>
    <td>{round(mse_new, 4)}</td>
    <td>{round(mse_new - mse_train, 4)}</td>
    <td>{round(100 * (mse_new - mse_train) / mse_train, 4)}</td>
    <td>5%</td>
    <td>{round(100 * (mse_new - mse_train) / mse_train, 4) > 5}</td>
  </tr>
</table>
"""

In [69]:
df_gt = df_gt.rename(columns={"remaining_lease_months": "remaining_lease"})
df_gt

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease,flat_type,storey_range,flat_model,district,predicted_price,actual_price,actual_price_scaled,predicted_price_scaled
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,380000.0,-0.584652,-0.996796
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,465000.0,-0.069426,-0.949282
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,370000.0,-0.645267,-1.081633
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,882888.0,2.463595,-0.822013
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,765000.0,1.749019,-1.051453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,585000.0,0.657952,-0.331979
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,673800.0,1.196212,-0.692508
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,725000.0,1.506559,-0.806234
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,1080000.0,3.658386,0.796600


### input drift - feature / label drift and concept drift

https://dataaspirant.com/kolmogorov-smirnov-test/
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

In [70]:
df_train = pd.read_csv("resale_flats_transformed_train.csv")
df_train.head(5)

,month,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,year,lat,lon,district,sale_date
0,4,3 ROOM,01 TO 03,67.0,New Generation,653,380000.0,2024,1.366980,103.855718,20.0,2024-04-01
1,4,3 ROOM,10 TO 12,73.0,New Generation,637,465000.0,2024,1.367487,103.849652,20.0,2024-04-01
2,4,3 ROOM,01 TO 03,68.0,New Generation,666,370000.0,2024,1.381562,103.840002,20.0,2024-04-01
3,4,4 ROOM,19 TO 21,94.0,Model A,1050,882888.0,2024,1.365483,103.844085,20.0,2024-04-01
4,4,4 ROOM,01 TO 03,93.0,Model A,1119,765000.0,2024,1.368864,103.833954,20.0,2024-04-01


### label drift

In [71]:
# label drift
# check if the distribution of the target variable has changed for df_train and df_gt

target_new = df_gt['actual_price']
target_train = df_train['resale_price']
ks_result = stats.kstest(target_new, target_train)
print(ks_result)
if ks_result.pvalue < 0.05:
    print("Label drift detected")
    print(f"p-value: {ks_result.pvalue}")
    # notify team

KstestResult(statistic=0.2529242551760516, pvalue=1.8140190273178546e-29, statistic_location=477888.0, statistic_sign=-1)
Label drift detected
p-value: 1.8140190273178546e-29


In [114]:
# html report for label drift, include ks test result

html_report_label_drift = f"""
<h3>Input Drift:</h3>
<h4>Label Drift</h4>
Resale Price
KS-test details: statistic={round(ks_result.statistic,4)}, p-value={round(ks_result.pvalue,4)}
{"Label drift detected !" if ks_result.pvalue < 0.05 else "No label drift detected"}
"""

### feature drift

#### numerical features

In [116]:
html_report_feature_drift = f"""
<h4>Feature Drift</h4>
"""

In [117]:
# feature drift - continuous
# check if the distribution of the features has changed for df_train and df_gt

num_cols = ['floor_area_sqm', 'remaining_lease']


for col in num_cols:
    ks_result = stats.kstest(df_gt[col], df_train[col])
    print(col, ks_result)
    html_report_feature_drift += f"""
    <h5>{col}</h5>
    KS-test details: statistic={round(ks_result.statistic,4)}, p-value={round(ks_result.pvalue,4)}
    {"Feature drift detected !" if ks_result.pvalue < 0.05 else "No feature drift detected"}
    """
    if ks_result.pvalue < 0.05:
        print(f"Feature drift detected for {col}")
        print(f"p-value: {ks_result.pvalue}")
        # notify team

floor_area_sqm KstestResult(statistic=0.043185933320604386, pvalue=0.2848595750286168, statistic_location=109.0, statistic_sign=1)
remaining_lease KstestResult(statistic=0.08256912921051834, pvalue=0.001698217067529511, statistic_location=769, statistic_sign=1)
Feature drift detected for remaining_lease
p-value: 0.001698217067529511


#### categorical features

In [120]:
# feature drift - categorical
cat_cols = ['flat_type', 'storey_range', 'flat_model', 'district']

# df_train sample
df_train_sample = df_train.sample(frac=0.2)

for col in cat_cols:
    df_gt_vc = df_gt[col].value_counts()
    df_train_vc = df_train_sample[col].value_counts()
    # ensure both value counts table have the same categories, if any missing then add the category with 0 count
    for cat in df_train_vc.index.unique():
        if cat not in df_gt_vc.index.unique():
            df_gt_vc.loc[cat] = 0
    for cat in df_gt_vc.index.unique():
        if cat not in df_train_vc.index.unique():
            df_train_vc.loc[cat] = 0

    # calculate the frequencies for both value counts table
    df_gt_vc = df_gt_vc / df_gt_vc.sum()
    df_train_vc = df_train_vc / df_train_vc.sum()

    print(df_gt_vc, df_train_vc)
    print(col, stats.chisquare(df_gt_vc, df_train_vc))
    print("\n")

    html_report_feature_drift += f"""
    <h5>{col}</h5>
    chi-square test details: statistic={round(stats.chisquare(df_gt_vc, df_train_vc).statistic,4)}, p-value={round(stats.chisquare(df_gt_vc, df_train_vc).pvalue,4)}
    {"Feature drift detected !" if stats.chisquare(df_gt_vc, df_train_vc).pvalue < 0.05 else "No feature drift detected"}
    """
    if stats.chisquare(df_gt_vc, df_train_vc).pvalue < 0.05:
        print(f"Feature drift detected for {col}")
        # notify team

flat_type
4 ROOM              0.45155
3 ROOM              0.24031
5 ROOM              0.23062
EXECUTIVE           0.04845
2 ROOM              0.02907
MULTI-GENERATION    0.00000
1 ROOM              0.00000
Name: count, dtype: float64 flat_type
4 ROOM              0.418138
5 ROOM              0.247415
3 ROOM              0.239199
EXECUTIVE           0.076720
2 ROOM              0.017679
MULTI-GENERATION    0.000623
1 ROOM              0.000227
Name: count, dtype: float64
flat_type Power_divergenceResult(statistic=0.02178907973237777, pvalue=0.9999997862393236)


storey_range
04 TO 06    0.242248
07 TO 09    0.207364
01 TO 03    0.182171
10 TO 12    0.143411
13 TO 15    0.102713
16 TO 18    0.058140
19 TO 21    0.032946
22 TO 24    0.013566
28 TO 30    0.005814
25 TO 27    0.003876
34 TO 36    0.003876
37 TO 39    0.001938
31 TO 33    0.001938
40 TO 42    0.000000
43 TO 45    0.000000
46 TO 48    0.000000
49 TO 51    0.000000
Name: count, dtype: float64 storey_range
04 TO 06    0.228660


In [121]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.image import MIMEImage

import pandas as pd
import matplotlib.pyplot as plt

from typing import Callable

# fill in your own email address and password
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_username = "bt4301group8@gmail.com"
smtp_password = "kbkpubaotefnlxpi"
sender_email = "bt4301group8@gmail.com"
receiver_email = "valentin_khoo@outlook.com"

In [122]:
subject = "ML Model Monitoring Report"
body = "Ground Truth Evaluation" + html_report_ground_truth_eval + html_report_label_drift + html_report_feature_drift

msg = MIMEMultipart()
msg["From"] = sender_email
msg["To"] = receiver_email
msg["Subject"] = subject

msg.attach(MIMEText(body, "html"))

In [123]:
with smtplib.SMTP(smtp_server, smtp_port) as server:
    try:
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email successfully sent")
    except:
        print("An exception occurred when sending the data profiling email...")

Email successfully sent
